# 20/05/2024 - Apply model to data

In [1]:

# NB! Only run this once
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\emily\Documents\repos\measuring-quality-of-life-gauteng


C:\Users\emily\AppData\Local\pypoetry\Cache\virtualenvs\measuring-quality-of-life-OxfVlamN-py3.10\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from tensorflow.keras.models import load_model
model = load_model("outputs/model/resnet.h5", compile=False)

In [4]:
import pandas as pd
data = pd.read_csv("outputs/model/data-split.csv")
len(data)

44300

In [5]:
data.head()

,Unnamed: 0,tile,qol_index,split
0,0,2018-04-1179-868_00_05.tiff,62.041629,train
1,1,2018-04-1179-868_00_06.tiff,63.701792,train
2,2,2018-04-1179-868_00_06.tiff,62.041629,test
3,3,2018-04-1179-868_00_07.tiff,63.701792,val
4,4,2018-04-1179-868_00_07.tiff,62.041629,train


In [9]:
test = data.groupby('tile').count()
test[test["split"] > 1]

,Unnamed: 0,qol_index,split
tile,,,
2018-04-1178-867_14_07.tiff,2,2,2
2018-04-1178-867_15_06.tiff,2,2,2
2018-04-1178-867_15_07.tiff,2,2,2
2018-04-1178-868_15_05.tiff,2,2,2
2018-04-1178-868_15_06.tiff,2,2,2
...,...,...,...
2021-01-1187-872_09_05.tiff,2,2,2
2021-01-1187-872_09_07.tiff,2,2,2
2021-01-1187-872_09_13.tiff,2,2,2


In [10]:
test[test["split"] == 1]

,Unnamed: 0,qol_index,split
tile,,,
2018-04-1178-866_14_00.tiff,1,1,1
2018-04-1178-866_14_01.tiff,1,1,1
2018-04-1178-866_14_02.tiff,1,1,1
2018-04-1178-866_14_03.tiff,1,1,1
2018-04-1178-866_15_00.tiff,1,1,1
...,...,...,...
2021-01-1189-873_07_05.tiff,1,1,1
2021-01-1189-873_07_06.tiff,1,1,1
2021-01-1189-873_08_04.tiff,1,1,1


In [11]:
grouped_data = data.groupby('tile').count()["split"] <= 1
grouped_data

tile
2018-04-1178-866_14_00.tiff    True
2018-04-1178-866_14_01.tiff    True
2018-04-1178-866_14_02.tiff    True
2018-04-1178-866_14_03.tiff    True
2018-04-1178-866_15_00.tiff    True
                               ... 
2021-01-1189-873_07_05.tiff    True
2021-01-1189-873_07_06.tiff    True
2021-01-1189-873_08_04.tiff    True
2021-01-1189-873_08_05.tiff    True
2021-01-1189-873_08_06.tiff    True
Name: split, Length: 31504, dtype: bool

In [12]:
selection = grouped_data[grouped_data].index.tolist()
unique_data = data[data["tile"].isin(selection)] 
unique_data

,Unnamed: 0,tile,qol_index,split
0,0,2018-04-1179-868_00_05.tiff,62.041629,train
5,5,2018-04-1179-868_00_08.tiff,63.701792,train
6,6,2018-04-1179-868_00_09.tiff,63.701792,train
7,7,2018-04-1179-868_00_10.tiff,63.701792,train
8,8,2018-04-1179-868_00_11.tiff,63.701792,val
...,...,...,...,...
44293,44293,2018-04-1184-870_15_10.tiff,61.696010,train
44294,44294,2018-04-1184-870_15_11.tiff,61.696010,train
44295,44295,2018-04-1184-870_15_12.tiff,61.696010,train
44296,44296,2018-04-1184-870_15_13.tiff,61.696010,test


In [14]:
test_data = unique_data[unique_data["split"] == 'test']
test_data

,Unnamed: 0,tile,qol_index,split
13,13,2018-04-1179-868_01_04.tiff,62.041629,test
36,36,2018-04-1179-868_02_08.tiff,62.041629,test
70,70,2018-04-1179-868_03_15.tiff,55.810812,test
118,118,2018-04-1179-868_05_14.tiff,55.810812,test
151,151,2018-04-1179-868_07_00.tiff,62.041629,test
...,...,...,...,...
44215,44215,2018-04-1184-870_12_06.tiff,63.490776,test
44262,44262,2018-04-1184-870_14_00.tiff,63.490776,test
44276,44276,2018-04-1184-870_14_11.tiff,61.696010,test
44287,44287,2018-04-1184-870_15_05.tiff,63.490776,test


# Applying model to test data

In [15]:
import numpy as np
from PIL import Image
import os
from tqdm import tqdm

folder = "outputs/tiles"
for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
    img_location = os.path.join(folder, row["tile"])  # create full path to image
    with Image.open(img_location) as img:  # open image with pillow

        img = np.array(img)
        img = img[:, :, :3]
        img = np.expand_dims(img, axis=0)
    predict = model.predict(img, verbose=0)
    test_data.loc[index,"prediction"] = predict[0][0]
    
test_data.head()    

  0%|          | 0/2188 [00:00<?, ?it/s]C:\Users\emily\AppData\Local\Temp\ipykernel_32548\2919003397.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[index,"prediction"] = predict[0][0]
100%|██████████| 2188/2188 [04:02<00:00,  9.01it/s]


,Unnamed: 0,tile,qol_index,split,prediction
13,13,2018-04-1179-868_01_04.tiff,62.041629,test,139.097244
36,36,2018-04-1179-868_02_08.tiff,62.041629,test,263.259735
70,70,2018-04-1179-868_03_15.tiff,55.810812,test,257.927734
118,118,2018-04-1179-868_05_14.tiff,55.810812,test,333.344971
151,151,2018-04-1179-868_07_00.tiff,62.041629,test,193.000931


In [16]:
len(test_data)

2188

In [17]:
test_data.to_csv("./temp-outputs/misc/predictions-test.csv", index=False)


The above was pretty wrong! There was no preprocessing and it was slow because of the for loop. Better to use the create_generator functionality.